<div style="float:right; color:red; font-weight:bold;">Rename this file before you work on it!</div>
# Pandas Spark Exercise 3

We use the [Adventure Works dataset](https://msdn.microsoft.com/en-us/library/hh403424.aspxto) to compare queries in
1. SQL
2. Python Pandas, and 
3. Spark DataFrames.


The data files are available on ARC (our Hadoop Cluster):

**Local File System**
<pre>
$ ls -l /home/data/AdventureWorks/
total 180
-rw-r--r-- 1 root data   6144 Sep  7 18:51 Customers.xls
-rw-r--r-- 1 root data 157184 Sep  7 18:51 Employees.xls
-rw-r--r-- 1 root data   7680 Sep  7 18:51 ItemsOrdered.xls
-rw-r--r-- 1 root data   5632 Sep  7 18:51 SalesTerritory.xls
</pre>

**HDFS**
<pre>
$ hdfs dfs -ls /user/pmolnar/data/AdventureWorks
Found 4 items
-rw-r--r--   3 pmolnar hadoop        466 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/Customer.csv.gz
-rw-r--r--   3 pmolnar hadoop      18125 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/Employees.csv.gz
-rw-r--r--   3 pmolnar hadoop        603 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/ItemsOrdered.csv.gz
-rw-r--r--   3 pmolnar hadoop        404 2017-02-07 22:46 /user/pmolnar/data/AdventureWorks/SalesTerritory.csv.gz
</pre>



## Task
1. Fill in the Python-Pandas or Python-Spark expressions to produce a table as described in the problem statements.
2. The SQL expression may give you a hint. It also allows you to see both systems side-by-side.

# Setup
## Pandas

In [ ]:
# %load adventureworks_pandas.py
import pandas as pd
import numpy as np
Empoyees = pd.read_excel('/home/data/AdventureWorks/Employees.xls')
Territory = pd.read_excel('/home/data/AdventureWorks/SalesTerritory.xls')
Customers = pd.read_excel('/home/data/AdventureWorks/Customers.xls')
Orders = pd.read_excel('/home/data/AdventureWorks/ItemsOrdered.xls')



## Spark

In [1]:
# %load pyspark_init.py
"""
Load packages and create context objects...
"""
import os
import platform
import sys
if not 'sc' in vars():
    sys.path.append('/usr/hdp/2.4.2.0-258/spark/python')
    os.environ["SPARK_HOME"] = '/usr/hdp/2.4.2.0-258/spark'
    os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.11:1.2.0 pyspark-shell'
    import py4j
    import pyspark
    from pyspark.context import SparkContext, SparkConf
    from pyspark.sql import SQLContext, HiveContext
    from pyspark.storagelevel import StorageLevel
    sc = SparkContext()
    import atexit
    atexit.register(lambda: sc.stop())
    print("""Welcome to
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version %s
          /_/
    """ % sc.version)
else:
    print("""Already running
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version %s
          /_/
    """ % sc.version)

if not 'sqlCtx' in vars():
    sqlCtx = SQLContext(sc)
print 'Spark Context available as `sc`'
print 'Spark SQL Context (%s) available as `sqlCtx`'%str(type(sqlCtx))
print "Monitor this application at http://arc.insight.gsu.edu:8088/proxy/"+sc.applicationId



Already running
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version 1.6.1
          /_/
    
Spark Context available as `sc`
Spark SQL Context (<class 'pyspark.sql.context.HiveContext'>) available as `sqlCtx`
Monitor this application at http://arc.insight.gsu.edu:8088/proxy/application_1484597252711_0267


In [5]:
# %load adventureworks_spark.py
if not 'sqlCtx' in vars():
   sqlCtx = SQLContext(sc)

Employees_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Employees.csv.gz')

Territory_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/SalesTerritory.csv.gz')

Orders_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/ItemsOrdered.csv.gz')

Customers_df = sqlCtx.read.format('com.databricks.spark.csv')\
    .options(header=True, inferschema=True)\
    .load('/user/pmolnar/data/AdventureWorks/Customer.csv.gz')



# Queries

### 1a. Provide a count of the number of salespeople in the company

### 1b. Instead of a count, list the salespeople along with their territories
Show columns: EmployeeID, FirstName, LastName, TerritoryName, CountryCode, Region

### 1c. Now include in the result set all other non-sales employees

### 1d. Sort the results with the salespeople at the top

### 1e. It turns out we don't need territory information for our American salespeople
Only show information for territories outside the U.S.

### 2a. Take the query from #1b above, but add information for each employee's manager
Show additional columns: ManagerFirstName, ManagerLastName, ManagerJobTitle

### 2b. Only show salespeople who report to Amy Alberts

### 3a. Give a list of all customers, and the average price of items ordered for each one
Show columns: CustomerID, FirstName, LastName, City, StateName, AvgPrice

### 3b. For those customers who have never ordered anything, put 0 as the average price

### 3c. Show the same results, but instead of average price per customer, tell me the average "Total Price" per customer
Make sure you change the alias of the AvgPrice column accordingly

### 4a. Someone has requested a report of all employees and their managers (if they don't have a manager, still display them)
Show columns: EmployeeID, EmployeeFullName, EmployeeTitle, BossFullName, BossTitle

### 4b. Another request from someone else has just come in, asking for a list of managers and their employees
Columns: BossFullName, BossTitle, EmployeeFullName, EmployeeTitle
Sort by BossTitle then EmployeeTitle

### 4c. Provide a list of managers in the company

### 5a. Show me only the salespeople who live in the U.S. and their hire dates
Columns: FullName, HireDate

### 5b. Show me only the salespeople who sell to U.S. customers and their hire dates, plus how much revenue their territory has generated this year
Columns: FullName, HireDate, TerritoryName, Revenue

### 6. I need a list of all the salespeople and their territories, but also show the territories that don't have any salespeople assigned.
Columns: EmployeeID, FirstName, LastName, TerritoryName, TerritoryCountry, TerritoryRegion

### 7. Give me a list of all the customers, and if they ordered anything less than $30 show me those orders too

### 9a. We need a list of each employee's boss's boss (i.e., 2 levels up)
Columns: EmployeeID, EmployeeFullName, EmployeeTitle, BossBossFullName, BossBossTitle<br/>
Sort by BossBossTitle and then EmployeeTitle

### 9b. Same as above, except show all employees in the company

### 9c. For those employees who don't have a boss's boss, display "n/a" for BossBossFullName and BossBossTitle

### 9d. Change the sort so that the ones with "n/a" are at the top of the list

### 10a. We need a list of each manager's indirect reports (i.e., 2 levels down)
Columns: ManagerFullName, ManagerTitle, IndirectReportFullName, IndirectReportTitle<br />
Sort by ManagerTitle and IndirectReportTitle

### 10b. Please tell us how many indirect reports each manager has
Columns: ManagerFullName, ManagerTitle, NumIndirectReports

### 11. Start with the query from #4c and change it so it returns just one number in the result: the count of managers in the company

### BONUS 1a. The HR people have come to you with a problem, they realized that some of the employee names were entered into the system with 2 spaces instead of 1
Find anyone with a FirstName, MiddleName, or LastName that has 2 spaces<br />
Columns: EmployeeID, FirstName, MiddleName, LastName

### BONUS 1b. Display the same list but correct the names to display what they should look like